# Jiani's data

In [4]:
# imports
import os
from pkg_resources import resource_filename
import numpy as np

from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C

import pandas

import lbg_da


from bokeh import plotting
from bokeh import models
from bokeh.models.glyphs import Patch

In [17]:
def set_fontsize(p, fsz):
    '''
    Parameters
    ----------
    ax : Bokeh plot class
    fsz : float
      Font size
    '''
    p.xaxis.axis_label_text_font_size = '{:d}pt'.format(fsz)
    p.xaxis.major_label_text_font_size = "{:d}pt".format(fsz)
    #
    p.yaxis.axis_label_text_font_size = '{:d}pt'.format(fsz)
    p.yaxis.major_label_text_font_size = "{:d}pt".format(fsz)

# Load

In [2]:
path = resource_filename('lbg_da', 'data/tau_eff')

In [3]:
dfile = os.path.join(path, 'dr14_result_alpha_deltatau.txt')

In [7]:
tbl = pandas.read_fwf(dfile)

In [8]:
tbl

,z,dtau
0,2.404482,-0.142293
1,2.405266,-0.136816
2,2.406050,-0.132280
3,2.406835,-0.128389
4,2.407619,-0.142128
5,2.408404,-0.137462
6,2.409189,-0.131403
7,2.409974,-0.131444
8,2.410759,-0.124143
9,2.411545,-0.115038


# Analyze $\Delta \tau$ first

## Prep

In [9]:
X = np.atleast_2d(tbl['z']).T
X.shape

(4512, 1)

In [10]:
# Remove the mean
mean_tau = np.mean(tbl['dtau'])
norm_tau = tbl['dtau'] - mean_tau

## GP

In [11]:
kernel = C(1.0, (1e-3, 1e3)) * RBF(0.1, (1e-2, 1e2))
gp = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=9,
                             alpha=0.01)

In [12]:
gp.fit(X, norm_tau)

GaussianProcessRegressor(alpha=0.01, copy_X_train=True,
                         kernel=1**2 * RBF(length_scale=0.1),
                         n_restarts_optimizer=9, normalize_y=False,
                         optimizer='fmin_l_bfgs_b', random_state=None)

In [21]:
x = np.atleast_2d(np.linspace(2., 5., 1000)).T
y_pred, sigma = gp.predict(x, return_std=True)

## Plot

In [24]:
# Plot
plotting.output_notebook()

p = plotting.figure(title='Dtau', x_axis_label='z',
                   y_axis_label='Dtau')#, y_range=(0., 1), x_range=(1.5,4.))

# Fill between

# GP
xv = x[:,0]

# Fill between
x2 = xv[::-1]
xp = np.hstack((xv, x2))
y2 = y_pred-sigma+mean_tau
y2 = y2[::-1]
yp = np.hstack((y_pred+sigma+mean_tau,y2))

source = models.ColumnDataSource(dict(x=xp, y=yp))
glyph = Patch(x="x", y="y", fill_color="blue")

p.add_glyph(source, glyph)


#p.circle(m19[0,:], m19[1,:], size=10, color="navy", alpha=0.9, legend='Monzon+19')
#
#for dset, ref, clr in zip([s03,k05,t17], ['Schaye', 'Kirkman', 'Thomas'], ['blue', 'red', 'green']):
#    p.circle(dset[0,:], dset[1,:], size=5, color=clr, alpha=0.5, legend=ref)
#    # Error bar
#    sdict = dict(base=dset[0,:], upper=dset[1,:]+dset[2,:], lower=dset[1,:]-dset[2,:])
#    source2 = models.ColumnDataSource(sdict)
#    whisk = models.Whisker(source=source2, base="base", upper="upper", lower="lower", level="overlay",
#                      line_color=clr, line_width=2)
#    p.add_layout(whisk)

# Ding
p.circle(tbl['z'], tbl['dtau'], size=5, color='gray', alpha=0.5, legend='Ding+19')
# Error bar
#sdict = dict(base=m19[0,:], upper=m19[1,:]+m19_sig, lower=m19[1,:]-m19_sig)
#source2 = models.ColumnDataSource(sdict)
#whisk = models.Whisker(source=source2, base="base", upper="upper", lower="lower", level="overlay",
#                  line_color='black', line_width=2)
#p.add_layout(whisk)

p.legend.location = "bottom_right"

# GP
p.line(x[:,0], y_pred+mean_tau, color='black', legend='Prediction')
#plt.fill(np.concatenate([x, x[::-1]]),
#         np.concatenate([y_pred - 1.9600 * sigma,
#                        (y_pred + 1.9600 * sigma)[::-1]]),
#         alpha=.5, fc='b', ec='None', label='95% confidence interval')

set_fontsize(p, 16)

plotting.show(p)

Loading BokehJS ...